In [8]:
import torch 
from d2l import torch as d2l

def corr2d_multi_in(X,K):
    """correlation 2d mutiple input"""
    return sum(d2l.corr2d(x,k)for x,k in zip(X,K))
    # zip(X,K)将输入X和卷积层K 的对应元素配对
    # 计算x和k之间的二维相关运算
    # X和K都是3d

    # 1v1 执行卷积运算，返回叠加和

In [9]:
X = torch.tensor([[[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]],
               [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]])
K = torch.tensor([[[0.0, 1.0], [2.0, 3.0]], [[1.0, 2.0], [3.0, 4.0]]])

corr2d_multi_in(X, K)

tensor([[ 56.,  72.],
        [104., 120.]])

In [10]:
def corr2d_multi_in_out(X,K):               # k是3d tensor 和完整的X进行互相关运算
    # correlation 2D multiple input and output
    # 在互相关运算中，每个输出通道先获取所有输出通道，再用对应该输出通道的卷积核计算出结果
    # 迭代K的第0个维度，每次都对输入X执行互相关运算
    # 最后把所有结果都叠加在一起
    #  # torch.stack 将所有通过 corr2d_multi_in(X, k) 计算的二维卷积结果沿着第0维进行堆叠
    return torch.stack([corr2d_multi_in(X,k)for k in K],0)
# K = torch.tensor([[[0.0, 1.0], [2.0, 3.0]], [[1.0, 2.0], [3.0, 4.0]]])
K=torch.stack((K,K+1,K+2),0)#确实是堆叠
K.shape,K

(torch.Size([3, 2, 2, 2]),
 tensor([[[[0., 1.],
           [2., 3.]],
 
          [[1., 2.],
           [3., 4.]]],
 
 
         [[[1., 2.],
           [3., 4.]],
 
          [[2., 3.],
           [4., 5.]]],
 
 
         [[[2., 3.],
           [4., 5.]],
 
          [[3., 4.],
           [5., 6.]]]]))

In [11]:
corr2d_multi_in_out(X, K)

tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])

In [13]:
"""全连接层实现1*1卷积"""
# 1*1卷积层需要的权重维度为co*ci再额外加一个偏置
def corr2d_multi_in_out_1x1(X, K):
    c_i, h, w = X.shape
    c_o = K.shape[0]
    X = X.reshape((c_i, h * w))
    K = K.reshape((c_o, c_i))
    # 全连接层中的矩阵乘法
    Y = torch.matmul(K, X)
    return Y.reshape((c_o, h, w))

In [ ]:
X = torch.normal(0, 1, (3, 3, 3))
K = torch.normal(0, 1, (2, 3, 1, 1))

Y1 = corr2d_multi_in_out_1x1(X, K)
Y2 = corr2d_multi_in_out(X, K)
assert float(torch.abs(Y1 - Y2).sum()) < 1e-6